In [1]:
import pandas as pd
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

csv_file = "/content/drive/MyDrive/JJIM/annotations1.csv"
df = pd.read_csv(csv_file)


image_folder = "/content/drive/MyDrive/JJIM/images"

for index, row in df.iterrows():
    image_name = row['Image Name']
    label = row['Image Name']

    image_path = os.path.join(image_folder, image_name)

    if os.path.exists(image_path):

        image = Image.open(image_path)

print(df)

KeyboardInterrupt: 

In [ ]:
!pip install git+https://github.com/jacobgil/pytorch-grad-cam.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.dropna(inplace=True)

In [ ]:
import random

image_files = os.listdir(image_folder)

random_image_files = random.sample(image_files, 5)

for image_file in random_image_files:
    image_path = os.path.join(image_folder, image_file)

    image = Image.open(image_path)

    width, height = image.size

    print(f"Image: {image_file}, Width: {width}, Height: {height}")

In [ ]:
class_counts = df['Majority Vote Label'].value_counts()

# class distribution
plt.figure(figsize=(6, 6))
sns.barplot(x=class_counts.index, y=class_counts.values, palette='viridis')
plt.title('Class Distribution in MHIST Dataset')
plt.xlabel('Majority Vote Label')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

# class imbalance
plt.figure(figsize=(6, 6))
sns.countplot(data=df, x='Majority Vote Label', palette='viridis')
plt.title('Class Imbalance Check')
plt.xlabel('Majority Vote Label')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
ssa_counts = df.loc[df['Majority Vote Label'] == 'SSA', 'Number of Annotators who Selected SSA Out of 7']
hp_counts = df.loc[df['Majority Vote Label'] == 'HP', 'Number of Annotators who Selected SSA Out of 7']

# histogram for distribution of Labels
plt.figure(figsize=(10, 6))
sns.histplot(ssa_counts, color='cyan', label='SSA', bins=8, alpha=0.7)
sns.histplot(hp_counts, color='blue', label='HP', bins=8, alpha=0.7)
plt.title('Distribution of SSA and HP Labels')
plt.xlabel('Number of Annotators who selected SSA out of 7')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
metadata_cols = ['Partition']

# cross-tabulation Labels
heatmap_data = pd.crosstab(df['Majority Vote Label'], df['Partition'])

# heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, fmt="d", cmap="Oranges")
plt.title('Class Distribution Across Train/Test Splits')
plt.xlabel('Partition')
plt.ylabel('Majority Vote Label')
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, roc_auc_score

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomDataset(Dataset):
    def __init__(self, df, image_folder, transform=None, partition='train'):
        self.df = df[df['Partition'] == partition]
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.df.iloc[idx, 0]
        img_path = os.path.join(self.image_folder, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = 1 if self.df.iloc[idx, 1] == 'SSA' else 0
        return image, label


train_dataset = CustomDataset(df, image_folder, transform, partition='train')
test_dataset = CustomDataset(df, image_folder, transform, partition='test')
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Vision Transformer model
class VisionTransformer(nn.Module):
    def __init__(self, num_classes):
        super(VisionTransformer, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(512 * 14 * 14, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

model1 = VisionTransformer(num_classes=2)  # 2 classes: SSA and HP

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model1.parameters(), lr=0.01, momentum=0.9)  # Using SGD with momentum

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = model1.cuda() if torch.cuda.is_available() else model1 # Move model to GPU if available

num_epochs = 15
for epoch in range(num_epochs):
    model1.train()
    running_loss = 0.0
    total_predictions = []
    true_labels = []

    for images, labels in train_loader:
        images = images.cuda() if torch.cuda.is_available() else images
        labels = labels.cuda() if torch.cuda.is_available() else labels

        optimizer.zero_grad()
        outputs = model1(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        _, predicted = torch.max(outputs, 1)
        total_predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy1 = accuracy_score(true_labels, total_predictions)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}, Accuracy: {epoch_accuracy1}")

model1.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.cuda() if torch.cuda.is_available() else images
        labels = labels.cuda() if torch.cuda.is_available() else labels

        outputs = model1(images)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

accuracy1 = correct_predictions / len(test_dataset)
print('\nTest accuracy:', accuracy1)

auc1 = roc_auc_score(true_labels, total_predictions)
print("AUC:", auc1)


In [ ]:
print(model1)

In [ ]:
import torch
import os
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
from pytorch_grad_cam import GradCAM
from PIL import Image
import matplotlib.pyplot as plt

target_layer = model1.model[9]

gradcam = GradCAM(model=model1, target_layers=[target_layer])

sample_indices = torch.randperm(len(test_dataset))[:5]

for idx in sample_indices:
    image, label = test_dataset[idx]
    image_tensor = image.unsqueeze(0)
    image_tensor = image_tensor.to(device)

    heatmap = gradcam(image_tensor)

    heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image.permute(1, 2, 0))
    plt.title('Original Image')
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(heatmap.squeeze(), cmap='jet', alpha=0.6)
    plt.title('GradCAM Heatmap')
    plt.axis('off')
    plt.show()



In [ ]:
import torchvision.models as models

class VisionTransformer(nn.Module):
    def __init__(self, num_classes):
        super(VisionTransformer, self).__init__()
        # pretrained ResNet-50 model
        self.model = models.resnet50(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

model2 = VisionTransformer(num_classes=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.0001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2 = model2.to(device)

num_epochs = 15
for epoch in range(num_epochs):
    model2.train()
    running_loss = 0.0
    total_predictions = []
    true_labels = []

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model2(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        _, predicted = torch.max(outputs, 1)
        total_predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy2 = accuracy_score(true_labels, total_predictions)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}, Accuracy: {epoch_accuracy2}")

model2.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model2(images)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

accuracy2 = correct_predictions / len(test_dataset)
print('\nTest accuracy:', accuracy2)

auc2 = roc_auc_score(true_labels, total_predictions)
print("AUC:", auc2)


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from pytorch_grad_cam import GradCAM
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

target_layer = model2.model.layer4[-1]

gradcam = GradCAM(model=model2, target_layers=[target_layer])

sample_indices = torch.randperm(len(test_dataset))[:5]

for idx in sample_indices:
    image, label = test_dataset[idx]
    image_tensor = image.unsqueeze(0).to(device)

    heatmap = gradcam(image_tensor)

    heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image.permute(1, 2, 0))
    plt.title('Original Image')
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(heatmap.squeeze(), cmap='jet', alpha=0.6)
    plt.title('GradCAM Heatmap')
    plt.axis('off')
    plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
from PIL import Image
import os

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, image_folder, transform=None):
        self.df = df
        self.image_folder = image_folder
        self.transform = transform
        self.label_encoder = LabelEncoder()

        self.df['Encoded_Label'] = self.label_encoder.fit_transform(self.df['Majority Vote Label'])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.df.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = torch.tensor(self.df.iloc[idx, -1], dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, label

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = CustomDataset(df, image_folder, transform=data_transforms)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# CNN model - ResNet50
class CNNResNet50(nn.Module):
    def __init__(self, num_classes):
        super(CNNResNet50, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        num_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.resnet50(x)

model3 = CNNResNet50(num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model3.parameters(), lr=0.0001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model3.to(device)

num_epochs = 15
for epoch in range(num_epochs):
    model3.train()
    running_loss = 0.0
    true_labels = []
    total_predictions = []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model3(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        _, predicted = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        total_predictions.extend(predicted.cpu().numpy())

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy3 = accuracy_score(true_labels, total_predictions)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}, Accuracy: {epoch_accuracy3}")

model3.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model3(images)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

accuracy3 = correct_predictions / len(test_loader.dataset)
print('\nTest accuracy:', accuracy3)

auc3 = roc_auc_score(true_labels, total_predictions)
print("AUC:", auc3)


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from pytorch_grad_cam import GradCAM
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

target_layer = model3.resnet50.layer4[-1]

gradcam = GradCAM(model=model3, target_layer=target_layer)

sample_indices = torch.randperm(len(test_dataset))[:5]

for idx in sample_indices:
    image, label = test_dataset[idx]
    image_tensor = image.unsqueeze(0).to(device)

    heatmap = gradcam(image_tensor)

    heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image.permute(1, 2, 0))
    plt.title('Original Image')
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(heatmap.squeeze(), cmap='jet', alpha=0.6)
    plt.title('GradCAM Heatmap')
    plt.axis('off')
    plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(epoch_accuracy1, label='Model 1 (Vision Transformer)')
plt.plot(epoch_accuracy2, label='Model 2 (Vision Transformer + Pretrained CNN)')
plt.plot(epoch_accuracy3, label='Model 3 (CNN)')
plt.title('Accuracy for Different Models')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(auc1, label='Model 1 (Vision Transformer)')
plt.plot(auc2, label='Model 2 (Vision Transformer + Pretrained CNN)')
plt.plot(auc3, label='Model 3 (CNN)')
plt.title('AUC for Different Models')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.legend()
plt.show()
